# Armoury Management System

# Done By: S. Vamsidhar

## Initializing the Packages

In [ ]:
import boto3
from botocore.exceptions import ClientError
from pprint import pprint
from decimal import Decimal
import time

## Connecting with DynamoDB

In [ ]:
client = boto3.client('dynamodb')

## Performing CRUD Operations

In [ ]:
def create():
    table = client.create_table(
        TableName = 'Armoury',
        KeySchema = [
            {
                'AttributeName': '_id',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'gun_type',
                'KeyType': 'RANGE'  # Sort key
            }
        ],
        AttributeDefinitions = [
            {
                'AttributeName': '_id',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'gun_type',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput = {
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table

In [ ]:
def show_menu():
    print("**********MENU**********")
    print("1. Insert gun details")
    print("2. Find a particular gun details with the help of primary keys")
    print("3. Update gun details")
    print("4. Delete a particular gun details")

    option = input("Enter option: ")
    return option

In [ ]:
def insert():
    
    print("**********ADD OPERATION**********")
    
    _id = input("Enter the ID of the gun: ")
    name1 = input("Enter the name of the gun: ")
    gun_type = input("Enter the type of the gun: ")
    country_of_origin = input("Enter the country of origin of the gun: ")
    ammo = input("Enter the details of the ammunition used for the gun: ")
    organizations_using = input("Enter the name of the organization using the gun: ")
    effective_firing_range = input("Enter the effective firing range of the gun: ")
    
    _id = int(_id)
    name1 = name1.lower()
    gun_type = gun_type.lower()
    country_of_origin = country_of_origin.lower()
    ammo = ammo.lower()
    organizations_using = organizations_using.lower()
    effective_firing_range = int(effective_firing_range)
    
    response = client.put_item(
       TableName = 'Armoury',
       Item = {
            '_id': {
                'N': "{}".format(_id),
            },
            'name1': {
                'S': "{}".format(name1),
            },
            'gun_type': {
                'S': "{}".format(gun_type),
            },
            'country_of_origin': {
                "S": "{}".format(country_of_origin),
            },
            'ammo': {
                "S": "{}".format(ammo),
            },
           'organizations_using': {
                "S": "{}".format(organizations_using),
            },
           'effective_firing_range': {
                "N": "{}".format(effective_firing_range), # In terms of meters
            }
        }
    )
    
    print('Record Added!!!')
    return response

In [ ]:
def get():
    
    print("**********RETRIEVAL OPERATION**********")
    
    _id = input("Enter the ID of the gun: ")
    gun_type = input("Enter the type of the gun: ")
    
    _id = int(_id)
    gun_type = gun_type.lower()
    
    try:
        response = client.get_item(       
                TableName = 'Armoury',
                Key = {
                        '_id': {
                                'N': "{}".format(_id),
                        },
                        'gun_type': {
                                'S': "{}".format(gun_type),
                        }
                    }
                )
        
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        pprint(response['Item'])
        return response['Item']

In [ ]:
def update():
    
    # AWS's official documentation says that we cannot update the primary key attributes. 
    # Instead, we can delete the item and create a new item with new attributes.
    
    # If the values are meant to remain the same, we just retype the existing values as it is.
    # Else the values will be treated as NULL if we skip the input stage of a particular field
    
    # So, first, we enter the id and type of the gun and then update its values.
    
    print("**********UPDATE OPERATION**********")
    
    _id = input("Enter the ID of the gun: ")
    name1 = input("Enter the name of the gun: ")
    gun_type = input("Enter the type of the gun: ")
    country_of_origin = input("Enter the country of origin of the gun: ")
    ammo = input("Enter the details of the ammunition used for the gun: ")
    organizations_using = input("Enter the name of the organization using the gun: ")
    effective_firing_range = input("Enter the effective firing range of the gun: ")
    
    _id = int(_id)
    name1 = name1.lower()
    gun_type = gun_type.lower()
    country_of_origin = country_of_origin.lower()
    ammo = ammo.lower()
    organizations_using = organizations_using.lower()
    effective_firing_range = int(effective_firing_range)
    
    response = client.update_item(
        TableName = 'Armoury',
        Key = {
            '_id': {
                    'N': "{}".format(_id),
            },
            'gun_type': {
                    'S': "{}".format(gun_type),
            }
        },
        ExpressionAttributeNames = {
            '#N': 'name1',
            '#C': 'country_of_origin',
            '#A': 'ammo',
            '#O': 'organizations_using',
            '#E': 'effective_firing_range'   
        },
        ExpressionAttributeValues = {
            ':n': {
                'S': "{}".format(name1),
            },
            ':c': {
                'S': "{}".format(country_of_origin),
            },
            ':a': {
                'S': "{}".format(ammo),
            },
            ':o': {
                'S': "{}".format(organizations_using),
            },
            ':e': {
                'N': "{}".format(effective_firing_range),
            }  
        },
        UpdateExpression = 'SET #N = :n, #C = :c, #A = :a, #O = :o, #E = :e',
        ReturnValues = "UPDATED_NEW"
    )
    
    print('Record Updated!!!')
    return response

In [ ]:
def delete():
    
    print("**********DELETE OPERATION**********")
    
    _id = input("Enter the ID of the gun: ")
    gun_type = input("Enter the type of the gun: ")
    name1 = input("Enter the name of the gun: ")
    
    _id = int(_id)
    gun_type = gun_type.lower()
    name1 = name1.lower()

    try:
        response = client.delete_item(
            TableName = 'Armoury',
            Key = {
                '_id': {
                    'N': "{}".format(_id),
                },
                'gun_type': {
                    'S': "{}".format(gun_type),
                }
            },
            ConditionExpression = "name1 = :n",
            ExpressionAttributeValues = {
                ':n': {
                    'S': "{}".format(name1),
                }
            }
        )
        
    except ClientError as e:
        if e.response['Error']['Code'] == "ConditionalCheckFailedException":
            print(e.response['Error']['Message'])
        else:
            raise
    else:
        print('Record Deleted!!!')
        return response 

In [ ]:
movie_table = create()
print("Create DynamoDB succeeded............")
print("Table status:{}".format(movie_table))

time.sleep(5)

In [ ]:
def main_loop():
    while True:
        option = show_menu()
        if option == "1":
            insert()
        elif option == "2":
            get()
        elif option == "3":
            update()
        elif option == "4":
            delete()
        elif option == "5":
            client.close()
            break
        else:
            print("Invalid option")
        print("")

In [ ]:
main_loop()